# DESTEK VEKTÖR MAKİNE ALGORİTMASI İLE MOBİL FİYAT SINIFLANDIRMASI

Cep telefonu kullanıcı bir telefon hizmet alanı içinde hareket ederken bir radyo frekansı bağlantısı üzerinden arama yapabilen ve alabilen taşınabilir bir telefondur.

1973 yılında Motorola'dan John F. Mitchell ve Martin Cooper tarafından tanıtılan ilk el tipi cep telefonu, **2 kilogram ağırlığında bir ahizeye** sahipti.

1979'da Nippon Telegraph and Telephone (NTT), Japonya'da dünyanın ilk hücresel ağını tanıttı. 1983'ten 2014'e kadar, dünya çapındaki cep telefonu abonelikleri yedi milyarın üzerine çıktı - bu dünyadaki her insan için en az bir tane cep telefonuna karşılık geliyordu.

2016'nın ilk çeyreğinde, dünya çapında en iyi akıllı telefon geliştiricileri Samsung, Apple ve Huawei idi ve akıllı telefon satışları toplam cep telefonu satışlarının yüzde 78'ini temsil etti.

source:https://en.wikipedia.org/wiki/Mobile_phone

Cep Telefonlarının Gelişimi
![](https://s3b.cashify.in/blog/wp-content/uploads/2018/05/mobile-phone-evolution.jpg)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")

Bu notebookta cep telefonu özelliklerini fiyat aralığı ile analiz ediyoruz. 
Ayrıca destek vektör makinesi (SVM) algoritmasını kullanarak bir model oluşturacağız.

## 1- VERİYİ OKUMA

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day6-SupportVectorMachines/notebooks"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART1/Day6-SupportVectorMachines/notebooks"

DATASET_PATH = ROOT_DIR + "/datasets/"

In [ ]:
data = pd.read_csv(DATASET_PATH + 'svm_mobile_price_train.csv')

In [ ]:
data.info()

In [ ]:
data.shape

21 öznitelik ve 2000 örneklem olan bir veri kümesidir.
Özniteliklerin anlamları aşağıda verilmiştir.
 

**battery_power:** Bir pilin bir seferde depolayabileceği toplam enerji mAh cinsinden ölçümü

**blue:** Bluetooth var veya yok

**clock_speed:** mikroişlemcinin talimatları yürütme hızı

**dual_sim:** Çift sim desteği var veya yok

**fc:** Ön Kamera mega piksel

**four_g:** 4G var veya yok

**int_memory:** Gigabayt cinsinden Dahili Bellek

**m_dep:** Mobil Derinlik, cm (Mobile Depth in cm)

**mobile_wt:** Cep telefonunun ağırlığı

**n_cores:** İşlemci çekirdek sayısı

**pc:** Birincil Kamera mega piksel

**px_height:** Piksel Çözünürlük Yüksekliği (Pixel Resolution Height)

**px_width:** Piksel Çözünürlük Genişliği (Pixel Resolution Width)

**ram:** Mega Bayt olarak RAM

**sc_h:** Cep telefonunun ekran yüksekliği cm olarak

**sc_w:** Cep telefonunun Ekran Genişliği, cm

**talk_time:** tek bir pil şarjının en uzun süre dayanacağı süre

**three_g:** 3G var veya yok

**touch_screen:** Dokunmatik ekrana sahip veya değil

**wifi:** Wifi var mı yok mu

**price_range:** Bu, 0(düşük maliyet), 1(orta maliyet), 2(yüksek maliyet) ve 3(çok yüksek maliyet) değerine sahip hedef değişkendir.


Aşağıdaki tabloda, ilk 10 örneklem gösterilmektedir.

In [ ]:
data.head(10)

Şimdi veri setinde eksik değerler olup olmadığını kontrol edelim.

In [ ]:
missing_values_count = data.isnull().sum()
missing_values_count


#### ÖZELLİKLER ARASINDAKİ KORELASYON

Aşağıdaki ısı haritası, özellikler arasındaki korelasyon değerlerini gösterir.Following heatmap shows correlation values between features.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr=data.corr()

fig = plt.figure(figsize=(15,12))
r = sns.heatmap(corr, cmap='Purples')
r.set_title("Correlation ")

**Görüldüğü gibi hedef fiyat aralığımız RAM ile oldukça pozitif bir korelasyona sahiptir.**
​

Ayrıca

* 3G ve 4G
* pc(Birincil Kamera mega piksel) ve fc(Ön Kamera mega piksel)
* px_weight(Piksel Çözünürlük Genişliği) ve px_height(Piksel Çözünürlük Yüksekliği)
* sc_w(Cebinin Ekran Genişliği cm olarak) ve sc_h(Cep telefonunun Ekran Yüksekliği cm olarak)
​
özelliklerin yüksek oranda pozitif korelasyonu vardır. 

Örneğin sc_w (mobilin ekran genişliği) arttıkça sc_h (mobilin ekran yüksekliği) artıyor.


Şimdi fiyat aralığı ve diğer özellikler arasındaki korelasyon değerlerini görelim.

In [ ]:
#price range correlation
corr.sort_values(by=["price_range"],ascending=False).iloc[0].sort_values(ascending=False)

## 3- DESTEK VEKTÖR MAKİNESİ (SUPPORT MACHINE VECTOR(SVM)) ALGORİTMASI

Destek vektör makineleri (SVM'ler), sınıflandırma, regresyon ve aykırı değerlerin tespiti için kullanılan bir dizi denetimli öğrenme yöntemidir.

Destek vektör makinelerinin avantajları şunlardır:

* Yüksek boyutlu uzaylarda etkilidir.
* Boyut sayısının örneklem sayısından fazla olduğu durumlarda hala etkilidir.
* Karar işlevinde (destek vektörleri olarak adlandırılır) eğitim noktalarının bir alt kümesini kullanır, bu nedenle bellek açısından da verimlidir.
* Çok yönlü: Karar işlevi için farklı Çekirdek işlevleri belirtilebilir. Ortak çekirdekler sağlanır, ancak özel çekirdekler belirtmek de mümkündür.

Destek vektör makinelerinin dezavantajları şunları içerir:

* Eğer öznitelik sayısı örnek sayısından çok fazla ise, Kernel fonksiyonlarını seçerken aşırı uydurmadan (overfit) kaçının ve regülerizasyon terimi çok önemlidir.
* SVM'ler doğrudan olasılık tahminleri sağlamaz, bunlar pahalı bir beş katlı çapraz doğrulama (five-fold cross-validation) kullanılarak hesaplanır.
 

source=(https://scikit-learn.org/stable/modules/svm.html)
                


#### EĞİTİM VE TEST KÜMESİNİN BÖLÜNMESİ

In [ ]:
y = data["price_range"].values

x = data.drop(["price_range"],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size = 0.2,random_state=1)

In [ ]:
#min-max scale
#x = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

#### Sınıf Dengesi Görselleştirme

Train-test split uyguladıktan sonra sınıfların nasıl bölündüğünü kontrol edelim.

In [ ]:
from yellowbrick.target import ClassBalance

visualizer = ClassBalance(labels=[0, 1, 2,3])
visualizer.fit(y_train, y_test)
visualizer.poof()

Tüm sınıf sayıları yaklaşık olarak aynıdır.

#### İLK MODEL

In [ ]:
from sklearn.svm import SVC

svm=SVC(random_state=1)

svm.fit(x_train,y_train)

print("train accuracy:",svm.score(x_train,y_train))
print("test accuracy:",svm.score(x_test,y_test))

Sklearn kütüphanesini uyguladıktan sonra, ilk model 0.97 eğitim doğruluğuna ve 0.87 test doğruluğuna sahiptir.

Ancak bu puanları nasıl artırabiliriz? Önce tahmincilerin doğruluk puanlarını iyileştirmek için gereksiz özellikleri azaltalım, ardından gridsearch yöntemini uygulayalım.


#### GRIDSEARCHCV İLE MODEL

Svm algoritmasını uygulamak için öncelikle parametreleri ayarlamak için GridSearchCV yöntemini kullandık. Bazı parametreler aşağıda verilmiştir.

In [ ]:
from sklearn.model_selection import GridSearchCV

C=[1,0.1,0.25,0.5,2,0.75]
kernel=["linear","rbf"]
gamma=["auto",0.01,0.001,0.0001,1]
decision_function_shape=["ovo","ovr"]

In [ ]:
svm=SVC(random_state=1)

grid_svm=GridSearchCV(estimator=svm,cv=5,param_grid=dict(kernel=kernel,C=C, gamma=gamma, decision_function_shape=decision_function_shape))
grid_svm.fit(x_train, y_train)

print("best score: ", grid_svm.best_score_)
print("best param: ", grid_svm.best_params_)

GridSearch'ü uyguladıktan sonra **C:2, decision_function_shape: "ovo", gama: "auto" ve kernel:"linear"** gibi en iyi parametrelere sahibiz.



#### CV PUANLARI

**StratifiedKFold:** Verileri eğitim/test setlerinde bölmek için eğitim/test endeksleri sağlar.

Bu çapraz doğrulama nesnesi, katmanlı kıvrımlar döndüren bir K-Fold çeşididir. Kıvrımlar, her sınıf için numune yüzdesi korunarak yapılır.

Aşağıdaki çubuk grafik, 10 kat ile SVC için Çapraz Doğrulama Puanlarını gösterir. Ve ortalama ortalama puan 0,957'dir.

In [ ]:

from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import CVScores
_, ax = plt.subplots()

# Create a cross-validation strategy
cv = StratifiedKFold(10)

# Create the cv score visualizer
oz = CVScores(
    SVC(C=2,decision_function_shape="ovo",gamma="auto",kernel="linear",random_state=1), ax=ax, cv=cv, scoring='accuracy'
)
oz.fit(x_train, y_train)
oz.poof()

#### MODELİ YAPILANDIRMAK

Gridsearch yöntemiyle bulduğumuz en iyi parametrelerle modeli oluşturalım.

In [ ]:
svm_model=SVC(C=2, decision_function_shape="ovo", gamma="auto", kernel="linear", random_state=1)

In [ ]:
svm_model.fit(x_train,y_train)

In [ ]:
print("train_accuracy:",svm_model.score(x_train,y_train))
print("test_accuracy: ", svm_model.score(x_test,y_test))

Modelimiz 0.98 eğitim doğruluğuna ve 0.96 test doğruluğuna sahiptir. Şimdi bunu karışıklık matrisi ile kontrol edelim.

In [ ]:
from yellowbrick.classifier import ConfusionMatrix

cm = ConfusionMatrix(
    svm_model, classes=[0,1,2,3]
)

cm.fit(x_train, y_train)
cm.score(x_test, y_test)

cm.poof()

Karışıklık matrisi modeline göre sarı kutuları yanlış biliyor.